In [30]:
%matplotlib inline
import math
import random
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats.stats import pearsonr

from sklearn import tree
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn import neighbors
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

In [31]:
def clean_data(df):
    df.replace({'Trim' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'SubModel' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'Color' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'Transmission' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'WheelTypeID' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'WheelType' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'Nationality' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'Size' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'TopThreeAmericanName' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRAcquisitionAuctionAveragePrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRAcquisitionAuctionCleanPrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRAcquisitionRetailAveragePrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRAcquisitonRetailCleanPrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRCurrentAuctionAveragePrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRCurrentAuctionCleanPrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRCurrentRetailAveragePrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'MMRCurrentRetailCleanPrice' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'PRIMEUNIT' : { np.nan : 'NS'}}, inplace=True)
    df.replace({'AUCGUART' : { np.nan : 'NS'}}, inplace=True)

    df['Transmission'] = df['Transmission'].str.upper()

    df = df.drop('PurchDate', axis=1)
    df = df.drop('VehYear',axis=1)
    df = df.drop('WheelTypeID', axis=1)
    df = df.drop('TopThreeAmericanName', axis=1)
    df = df.drop('BYRNO', axis=1)

    auctions = sorted(df['Auction'].unique())
    vehicleAges = sorted(df['VehicleAge'].unique())
    makes = sorted(df['Make'].unique())
    models = sorted(df['Model'].unique())
    trims = sorted(df['Trim'].unique())
    subModels = sorted(df['SubModel'].unique())
    colors = sorted(df['Color'].unique())
    transmissions = sorted(df['Transmission'].unique())
    wheeltypes = sorted(df['WheelType'].unique())
    vehOdos = sorted(df['VehOdo'].unique())
    nationalities = sorted(df['Nationality'].unique())
    sizes = sorted(df['Size'].unique())
    demands = sorted(df['PRIMEUNIT'].unique())
    guarantees = sorted(df['AUCGUART'].unique())
    zipcodes = sorted(df['VNZIP1'].unique())    #VNZIP1
    states = sorted(df['VNST'].unique())        #VNST
    vehcosts = sorted(df['VehBCost'].unique())
    onlinesales = sorted(df['IsOnlineSale'].unique())
    warranties = sorted(df['WarrantyCost'].unique())
    
    auctionsDictionary = dict(zip(auctions,range(0, len(auctions)+1)))
    makesDictionary = dict(zip(makes,range(0, len(makes)+1)))
    modelsDictionary = dict(zip(models,range(0, len(models)+1)))
    trimsDictionary = dict(zip(trims,range(0, len(trims)+1)))
    subModelsDictionary = dict(zip(subModels,range(0, len(subModels)+1)))
    colorsDictionary = dict(zip(colors,range(0, len(colors)+1)))
    transmissionsDictionary = dict(zip(transmissions,range(0, len(transmissions)+1)))
    wheeltypesDictionary = dict(zip(wheeltypes,range(0, len(wheeltypes)+1)))
    nationalitiesDictionary = dict(zip(nationalities,range(0, len(nationalities)+1)))
    sizesDictionary = dict(zip(sizes,range(0, len(sizes)+1)))
    demandsDictionary = dict(zip(demands,range(0, len(demands)+1)))
    guaranteesDictionary = dict(zip(guarantees,range(0, len(guarantees)+1)))
    vehcostsDictionary = dict(zip(vehcosts,range(0, len(vehcosts)+1)))  #not object but mapped as dictionary
    zipcodesDictionary = dict(zip(zipcodes,range(0, len(zipcodes)+1)))  #not object but mapped as dictionary
    statesDictionary = dict(zip(states,range(0, len(states)+1)))
    warrantiesDictionary = dict(zip(warranties,range(0, len(warranties)+1)))
    
    df['AuctionVal'] = df['Auction'].map(auctionsDictionary)
    df['MakeVal'] = df['Make'].map(makesDictionary)
    df['ModelVal'] = df['Model'].map(modelsDictionary)
    df['TrimVal'] = df['Trim'].map(trimsDictionary)
    df['SubModelVal'] = df['SubModel'].map(subModelsDictionary)
    df['ColorVal'] = df['Color'].map(colorsDictionary)
    df['TransmissionVal'] = df['Transmission'].map(transmissionsDictionary)
    df['WheelTypeVal'] = df['WheelType'].map(wheeltypesDictionary)
    df['NationalityVal'] = df['Nationality'].map(nationalitiesDictionary)
    df['SizeVal'] = df['Size'].map(sizesDictionary)
    df['PRIMEUNITVal'] = df['PRIMEUNIT'].map(demandsDictionary)
    df['AUCGUARTVal'] = df['AUCGUART'].map(guaranteesDictionary)
    df['VehBCostVal'] = df['VehBCost'].map(vehcostsDictionary)
    df['VNZIP1Val'] = df['VNZIP1'].map(zipcodesDictionary)
    df['VNSTVal'] = df['VNST'].map(statesDictionary)
    df['WarrantyCostVal'] = df['WarrantyCost'].map(warrantiesDictionary)

    dfFeatureSelection = df.drop(['RefId','MMRAcquisitionAuctionAveragePrice','MMRAcquisitionAuctionCleanPrice','MMRAcquisitionRetailAveragePrice',
                                 'MMRAcquisitonRetailCleanPrice','MMRCurrentAuctionAveragePrice','MMRCurrentAuctionCleanPrice','MMRCurrentRetailAveragePrice',
                                 'MMRCurrentRetailCleanPrice'], axis=1)

    dfFeatureSelection = dfFeatureSelection.drop(['Auction','Make','Model','Trim','SubModel','Color','Transmission','WheelType','Nationality','Size',
                                  'PRIMEUNIT','AUCGUART','VehBCost','VNZIP1','VNST','WarrantyCost'], axis=1)

    nAttributesRanking = len(dfFeatureSelection.columns)
    
    return df

In [32]:
df = pd.read_csv("training.csv") 
df2 = pd.read_csv("test.csv")

test_id = df2['RefId']

df = clean_data(df)
df2 = clean_data(df2)

# Divide the array in two
#df_goodbuy = df[df['IsBadBuy'].map(lambda x: x == 0)]
#df_badbuy = df[df['IsBadBuy'].map(lambda x: x == 1)]

#df_goodbuy.drop(df_goodbuy.index[[1,3]], inplace=True)
#df_goodbuy = df_goodbuy.sample(frac=1)
#df_goodbuy = df_goodbuy[:][:len(df_badbuy)]

#frames = [df_goodbuy, df_badbuy]
#df = pd.concat(frames)

/Users/Italo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
df_train = pd.concat([
    df['IsBadBuy'],
    df['VehicleAge'],
    df['VehOdo'],
    #df['IsOnlineSale'],
    df['AuctionVal'],
    df['MakeVal'],
    #df['ModelVal'],
    #df['TrimVal'],
    #df['SubModelVal'],
    #df['ColorVal'],
    #df['TransmissionVal'],
    df['WheelTypeVal'],
    df['NationalityVal'],
    #df['SizeVal'],
    #df['PRIMEUNITVal'],
    #df['AUCGUARTVal'],
    df['VehBCostVal'],
    #df['VNZIP1Val'],
    df['VNSTVal'],
    df['WarrantyCostVal']
    ],
    axis=1)

df_test = pd.concat([
    df2['VehicleAge'],
    df2['VehOdo'],
    #df2['IsOnlineSale'],
    df2['AuctionVal'],
    df2['MakeVal'],
    #df2['ModelVal'],
    #df2['TrimVal'],
    #df2['SubModelVal'],
    #df2['ColorVal'],
    #df2['TransmissionVal'],
    df2['WheelTypeVal'],
    df2['NationalityVal'],
    #df2['SizeVal'],
    #df2['PRIMEUNITVal'],
    #df2['AUCGUARTVal'],
    df2['VehBCostVal'],
    #df2['VNZIP1Val'],
    df2['VNSTVal'],
    df2['WarrantyCostVal']
    ],
    axis=1)

In [34]:
#Convert the DataFrame to a numpy array:
train_data = df_train.values

In [35]:
# Training data features, skip the first column 'Survived'
train_features = train_data[:, 1:]

# 'Survived' column values
train_target = train_data[:, 0]

test_features = df_test.values

In [36]:
train_features[0]

array([    3, 53973,     1,     4,     1,     0,   808,     5,    67])

In [37]:
train_target[1]

0

In [38]:
# Fit the model to our training data
clf = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, 
                                  min_samples_split=2, min_samples_leaf=2)
clf = clf.fit(train_features, train_target)

In [39]:
clf.feature_importances_

array([ 0.0870657 ,  0.23553514,  0.02891212,  0.05981487,  0.15886905,
        0.00509981,  0.22535592,  0.0998303 ,  0.0995171 ])

In [40]:
pred_target = clf.predict(train_features)

print metrics.precision_score(train_target, pred_target, average='weighted')
print metrics.recall_score(train_target, pred_target, average='weighted')
print metrics.f1_score(train_target, pred_target, average='weighted')
print metrics.accuracy_score(train_target, pred_target)
print metrics.precision_recall_fscore_support(train_target, pred_target)

0.934252561647
0.930759803922
0.930620296111
0.930759803922
(array([ 0.89530723,  0.97319789]), array([ 0.9756016,  0.885918 ]), array([ 0.93373141,  0.92750919]), array([8976, 8976]))


In [41]:
len(train_data)

17952

In [42]:
clf = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=7, 
                                  min_samples_split=2, min_samples_leaf=10) #, class_weight='balanced')

clf = tree.DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=288500278, splitter='best')

clf = RandomForestClassifier(n_estimators=30, 
                             criterion='gini',
                             max_features=None,
                             max_depth=None, 
                             min_samples_split=2,
                             min_samples_leaf=5,
                             bootstrap=True,
                             oob_score=False,
                             random_state=None,
                             class_weight=None) #balanced

clf = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=4, 
                                  #min_samples_split=2, min_samples_leaf=10, 
                                  class_weight=None # 'balanced'
                                 )

clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)



clf=clf.fit(train_features, train_target)

In [43]:
train_pred = clf.predict(train_features)
test_pred = clf.predict(test_features)

test = np.arange(73015, 121722, dtype=np.int)
tmp = pd.concat([
    pd.DataFrame(test_id),
    pd.DataFrame(test_pred)
    ],
    axis=1)

tmp.to_csv("foo.csv", sep=",", index=False)